### Bug: procs applied to `y_names` fields
The desired behavior is to not apply `procs` to `y_names` fields. These procs are input to `TabularDataLoaders.from_df`

This occurs in tabular.data.py line19:
    
    if cont_names is None: cont_names = list(set(df)-set(cat_names)-set(y_names))


In [13]:
from fastai.tabular.all import *
import pandas as pd
import torch

In [15]:
import fastai
fastai.__version__

'2.0.13'

### Demonstrate Issue

In [2]:
d = torch.randn((100, 6))
df = pd.DataFrame(d, columns=[f'c{i}' for i in range(d.shape[1])])
df = df + 10

#### By not enclosing y_names as a list, it gets Normalized

In [6]:
dls = TabularDataLoaders.from_df(df,
                                 procs=[Normalize],
                                 y_names='c5',)

dls.one_batch()[2].min()

tensor(-3.4584)

In [8]:
dls.ys[:3]

,c5
48,0.578217
56,-0.867873
13,-0.150244


##### By setting y_names as a list, it doesn't get Normalized

In [10]:
dls = TabularDataLoaders.from_df(df,
                                 procs=[Normalize],
                                 y_names=['c5'],)

dls.one_batch()[2].min()

tensor(6.6121)

In [11]:
dls.ys[:3]

,c5
97,10.411242
91,11.624607
21,10.472753


### Show how code modification works

#### Simple python gotcha 

In [30]:
y_names = 'column1'
set(y_names)

{'1', 'c', 'l', 'm', 'n', 'o', 'u'}

In [31]:
set(['column1', 'column2']) - set('column1')

{'column1', 'column2'}

#### Old code from library
`cont_names` does not remove c3 and c4 when `y_names` and `cat_names` are passed as strings instead of lists.

In [24]:
df = pd.DataFrame({'c1':[1,2], 'c2':[2,3], 'c3':[4,5], 'c4':[5,6]})

In [27]:
# inputs
cont_names = None
cat_names = 'c3'
y_names = 'c4'

# source code
if cat_names is None: cat_names = []
if cont_names is None: cont_names = list(set(df)-set(cat_names)-set(y_names))

# results
cont_names, cat_names, y_names

(['c2', 'c4', 'c1', 'c3'], 'c3', 'c4')

#### PR changed code

In [28]:
df = pd.DataFrame({'c1':[1,2], 'c2':[2,3], 'c3':[4,5], 'c4':[5,6]})

In [29]:
#inputs
cont_names = None
cat_names = 'c3'
y_names = 'c4'

# source code
if cat_names is None: cat_names = []
if cont_names is None: cont_names = list(set(df)-set(L(cat_names))-set(L(y_names)))

# reesults
cont_names, cat_names, y_names

(['c2', 'c1'], 'c3', 'c4')